In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# model parameters
reuse = False
block = MEGNet_block
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA.pickle'
batch_size = 32
dim = 128
epochs = 50
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4

In [3]:
prefix = '_MEGNet_5folds.csv'

In [4]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [5]:
folds = []
for f in range(5):
    with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(f)+'.pickle', 'rb') as handle:
        folds.append(pickle.load(handle))
folds = [[Data(**d) for d in fold] for fold in folds]

In [7]:
for i in range(5):
    print('\nstart fold '+str(i))
    # parpare data
    train_list = []
    val_list = []
    for j in range(5):
        if i == j:
            val_list.extend(folds[j])
        else:
            train_list.extend(folds[j])
    
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    val_dl = DataLoader(val_list,batch_size,shuffle=False)
    
    # train model
    model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,scheduler=scheduler)
    
    # predict oof for each type
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'_id.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # load model
        model.load_state_dict(bestWeight[type_i])
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        submit_ = dict(zip(test_id,yhat))
        train['fold'+str(i)+'_type'+str(type_i)] = train.id.map(submit_)
    
    # predict test
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('test').split('pickle')[0][:-1]+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('test').split('pickle')[0][:-1]+'_id_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # load model
        model.load_state_dict(bestWeight[type_i])
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        submit_ = dict(zip(test_id,yhat))
        test['fold'+str(i)+'_type'+str(type_i)] = test.id.map(submit_)


start fold 0
epoch:0, train_loss: +0.358, val_loss: -0.051, 
train_vector: +4.22|+0.96|+0.12|-0.42|-0.57|+0.11|-0.61|-0.93, 
val_vector  : +3.92|+0.45|-0.28|-1.01|-1.10|-0.20|-1.03|-1.16

epoch:1, train_loss: -0.298, val_loss: -0.715, 
train_vector: +2.25|+0.34|-0.33|-0.96|-1.07|-0.25|-1.05|-1.31, 
val_vector  : +0.77|-0.22|-0.52|-1.43|-1.23|-0.37|-1.27|-1.45

epoch:2, train_loss: -0.629, val_loss: -0.569, 
train_vector: +0.89|+0.12|-0.51|-1.18|-1.27|-0.38|-1.21|-1.50, 
val_vector  : +0.60|-0.13|-0.59|-0.82|-0.84|-0.44|-0.88|-1.44

epoch:3, train_loss: -0.728, val_loss: -0.853, 
train_vector: +0.81|-0.04|-0.60|-1.31|-1.33|-0.46|-1.30|-1.60, 
val_vector  : +0.56|-0.38|-0.68|-1.30|-1.54|-0.54|-1.29|-1.66

epoch:4, train_loss: -0.825, val_loss: -0.949, 
train_vector: +0.77|-0.15|-0.69|-1.42|-1.49|-0.53|-1.38|-1.70, 
val_vector  : +0.17|+0.01|-0.75|-1.60|-1.62|-0.59|-1.44|-1.77

epoch:5, train_loss: -0.893, val_loss: -0.926, 
train_vector: +0.71|-0.23|-0.76|-1.50|-1.56|-0.59|-1.44|-1.77, 

epoch:47, train_loss: -1.833, val_loss: -1.780, 
train_vector: -0.46|-1.35|-1.51|-2.46|-2.59|-1.24|-2.26|-2.78, 
val_vector  : -0.36|-1.38|-1.50|-2.54|-2.44|-1.25|-2.23|-2.54

epoch:48, train_loss: -1.845, val_loss: -1.793, 
train_vector: -0.48|-1.37|-1.52|-2.46|-2.61|-1.25|-2.27|-2.80, 
val_vector  : -0.53|-1.45|-1.52|-2.34|-2.44|-1.25|-2.27|-2.56

epoch:49, train_loss: -1.852, val_loss: -1.726, 
train_vector: -0.49|-1.37|-1.52|-2.48|-2.61|-1.25|-2.27|-2.81, 
val_vector  : -0.47|-0.92|-1.51|-2.46|-2.42|-1.25|-2.21|-2.58

Training completed in 4053.6233603954315s

start fold 1
epoch:0, train_loss: +0.446, val_loss: +0.089, 
train_vector: +4.35|+1.33|+0.17|-0.38|-0.55|+0.14|-0.55|-0.95, 
val_vector  : +4.14|+1.29|-0.26|-0.90|-0.96|-0.18|-1.07|-1.33

epoch:1, train_loss: -0.144, val_loss: -0.540, 
train_vector: +3.54|+0.37|-0.34|-0.96|-1.10|-0.25|-1.06|-1.36, 
val_vector  : +1.07|+0.15|-0.47|-0.99|-1.18|-0.33|-1.14|-1.44

epoch:2, train_loss: -0.619, val_loss: -0.656, 
train_vector: +0.9

epoch:44, train_loss: -1.886, val_loss: -1.792, 
train_vector: -0.51|-1.50|-1.49|-2.52|-2.70|-1.23|-2.28|-2.86, 
val_vector  : -0.53|-1.37|-1.48|-2.40|-2.46|-1.24|-2.27|-2.60

epoch:45, train_loss: -1.892, val_loss: -1.786, 
train_vector: -0.52|-1.50|-1.50|-2.52|-2.72|-1.23|-2.29|-2.87, 
val_vector  : -0.54|-1.17|-1.48|-2.49|-2.49|-1.23|-2.28|-2.60

epoch:46, train_loss: -1.898, val_loss: -1.640, 
train_vector: -0.52|-1.52|-1.50|-2.53|-2.72|-1.23|-2.29|-2.88, 
val_vector  : -0.21|-0.42|-1.48|-2.49|-2.47|-1.23|-2.24|-2.59

epoch:47, train_loss: -1.904, val_loss: -1.710, 
train_vector: -0.53|-1.52|-1.50|-2.53|-2.73|-1.24|-2.30|-2.88, 
val_vector  : -0.38|-0.80|-1.43|-2.56|-2.49|-1.22|-2.22|-2.58

epoch:48, train_loss: -1.911, val_loss: -1.690, 
train_vector: -0.53|-1.54|-1.51|-2.54|-2.74|-1.24|-2.30|-2.89, 
val_vector  : -0.15|-0.80|-1.50|-2.37|-2.54|-1.24|-2.31|-2.60

epoch:49, train_loss: -1.912, val_loss: -1.679, 
train_vector: -0.53|-1.53|-1.51|-2.54|-2.74|-1.24|-2.30|-2.90, 
val_vec

epoch:41, train_loss: -1.627, val_loss: -1.468, 
train_vector: -0.26|-0.98|-1.38|-2.25|-2.34|-1.16|-2.09|-2.56, 
val_vector  : +0.96|-1.11|-1.39|-2.31|-2.20|-1.18|-2.10|-2.42

epoch:42, train_loss: -1.634, val_loss: -1.582, 
train_vector: -0.25|-0.99|-1.39|-2.26|-2.34|-1.16|-2.10|-2.56, 
val_vector  : -0.12|-0.85|-1.40|-2.37|-2.19|-1.18|-2.11|-2.43

epoch:43, train_loss: -1.643, val_loss: -1.598, 
train_vector: -0.27|-1.00|-1.40|-2.26|-2.36|-1.17|-2.11|-2.58, 
val_vector  : -0.15|-1.06|-1.37|-2.26|-2.17|-1.20|-2.14|-2.43

epoch:44, train_loss: -1.652, val_loss: -1.588, 
train_vector: -0.28|-1.03|-1.40|-2.26|-2.36|-1.18|-2.12|-2.58, 
val_vector  : -0.51|-0.86|-1.40|-1.96|-2.25|-1.18|-2.13|-2.41

epoch:45, train_loss: -1.665, val_loss: -1.562, 
train_vector: -0.30|-1.05|-1.41|-2.27|-2.38|-1.18|-2.13|-2.60, 
val_vector  : +0.03|-0.76|-1.42|-2.40|-2.12|-1.20|-2.18|-2.45

epoch:46, train_loss: -1.665, val_loss: -1.575, 
train_vector: -0.30|-1.06|-1.41|-2.27|-2.37|-1.19|-2.13|-2.59, 
val_vec

epoch:38, train_loss: -1.754, val_loss: -1.667, 
train_vector: -0.37|-1.28|-1.42|-2.39|-2.52|-1.17|-2.18|-2.70, 
val_vector  : -0.14|-1.04|-1.43|-2.48|-2.37|-1.18|-2.20|-2.50

epoch:39, train_loss: -1.760, val_loss: -1.589, 
train_vector: -0.38|-1.28|-1.43|-2.40|-2.53|-1.18|-2.19|-2.71, 
val_vector  : -0.38|-0.93|-1.43|-2.22|-2.13|-1.18|-1.97|-2.48

epoch:40, train_loss: -1.854, val_loss: -1.834, 
train_vector: -0.47|-1.49|-1.46|-2.50|-2.64|-1.20|-2.25|-2.82, 
val_vector  : -0.54|-1.55|-1.47|-2.56|-2.49|-1.21|-2.27|-2.59

epoch:41, train_loss: -1.874, val_loss: -1.825, 
train_vector: -0.49|-1.52|-1.48|-2.52|-2.68|-1.21|-2.27|-2.83, 
val_vector  : -0.57|-1.44|-1.47|-2.58|-2.49|-1.21|-2.25|-2.59

epoch:42, train_loss: -1.885, val_loss: -1.775, 
train_vector: -0.49|-1.54|-1.48|-2.53|-2.70|-1.21|-2.28|-2.86, 
val_vector  : -0.40|-1.31|-1.47|-2.52|-2.45|-1.22|-2.24|-2.59

epoch:43, train_loss: -1.889, val_loss: -1.826, 
train_vector: -0.50|-1.53|-1.48|-2.53|-2.71|-1.22|-2.28|-2.87, 
val_vec

epoch:35, train_loss: -1.568, val_loss: -1.447, 
train_vector: -0.18|-0.95|-1.35|-2.17|-2.28|-1.12|-2.03|-2.48, 
val_vector  : +0.26|-0.55|-1.39|-2.00|-2.24|-1.15|-2.10|-2.39

epoch:36, train_loss: -1.575, val_loss: -1.498, 
train_vector: -0.17|-0.96|-1.36|-2.18|-2.28|-1.12|-2.04|-2.49, 
val_vector  : -0.12|-0.36|-1.40|-2.31|-2.20|-1.17|-2.07|-2.36

epoch:37, train_loss: -1.595, val_loss: -1.588, 
train_vector: -0.23|-1.00|-1.36|-2.18|-2.30|-1.13|-2.06|-2.50, 
val_vector  : -0.38|-0.83|-1.41|-2.24|-2.25|-1.16|-2.07|-2.37

epoch:38, train_loss: -1.597, val_loss: -1.530, 
train_vector: -0.22|-0.97|-1.37|-2.20|-2.31|-1.14|-2.06|-2.51, 
val_vector  : +0.33|-1.22|-1.37|-2.08|-2.22|-1.17|-2.09|-2.41

epoch:39, train_loss: -1.743, val_loss: -1.767, 
train_vector: -0.36|-1.24|-1.45|-2.36|-2.50|-1.19|-2.18|-2.67, 
val_vector  : -0.58|-1.39|-1.50|-2.26|-2.41|-1.24|-2.24|-2.51

epoch:40, train_loss: -1.773, val_loss: -1.802, 
train_vector: -0.38|-1.28|-1.46|-2.39|-2.53|-1.21|-2.20|-2.71, 
val_vec

In [8]:
assert set(test.iloc[:,5:].isnull().sum(1)) == set([7*5])
test['yhat'] = np.nanmean(test.iloc[:,5:],1)
test = test[['id','yhat']]
test.to_csv('../Data/test'+prefix,index=False)

assert set(train.iloc[:,6:].isnull().sum(1)) == set([train.iloc[:,6:].shape[1]-1])
train['yhat'] = np.nanmean(train.iloc[:,6:],1)
train = train[['id','yhat']]
train.to_csv('../Data/train'+prefix,index=False)